## Modelltraining

In [43]:
import json
import spacy
from spacy.tokens import DocBin


### Datenkonvertierung

In [45]:
with open("training_data_fixed_.jsonl", "r", encoding="utf-8") as f:
    training_data = [json.loads(line) for line in f if line.strip()]

In [46]:
nlp = spacy.blank("de")

db = DocBin()

for item in training_data:
    text, annotations = item[0], item[1]

    doc = nlp.make_doc(text)

    ents = []
    for start, end, label in annotations:

        span = doc.char_span(start, end, label=label, alignment_mode="contract")
        if span is None:
            
            raise ValueError(
                f"Kann Span nicht bilden")
        ents.append(span)


    doc.ents = ents
    db.add(doc)

db.to_disk("output/train.spacy")


ValueError: Kann Span nicht bilden

In [27]:
# Welches Token ist "4"?
print(doc[33].text)

# Zeig alle Spans, die Token 4 enthalten
conflicts = [(s.text, s.start, s.end, s.label_) for s in ents if s.start <= 33 < s.end]
print(conflicts)


Mar
[('Mar. Dor. Seidelin', 33, 38, 'PER'), ('Mar.', 33, 35, 'DATE')]


In [47]:
import spacy
from spacy.tokens import DocBin
from spacy.util import filter_spans

nlp = spacy.blank("de")
db = DocBin()

def make_spans(doc, annotations, ex_id=""):
    made = []
    for (start, end, label) in annotations:
        # 1) Basis-Checks
        if not (0 <= start < end <= len(doc.text)):
            raise ValueError(f"[{ex_id}] Ungültige Offsets: {(start, end, label)} | len(text)={len(doc.text)}")

        # 2) Erst 'contract', dann Fallback 'expand'
        span = doc.char_span(start, end, label=label, alignment_mode="contract")
        if span is None:
            span = doc.char_span(start, end, label=label, alignment_mode="expand")

        # 3) Wenn immer noch None: präzise Fehlermeldung
        if span is None:
            snippet = doc.text[max(0, start-25):min(len(doc.text), end+25)].replace("\n", " ")
            frag = doc.text[start:end]
            raise ValueError(
                f"[{ex_id}] Kann Span nicht bilden für {frag!r} [{start},{end}) {label}. "
                f"Kontext: …{snippet}…"
            )
        made.append(span)

    # 4) Duplikate raus + Überlappungen filtern (behält längste Spans)
    uniq = {(s.start, s.end, s.label_): s for s in made}.values()
    cleaned = filter_spans(list(uniq))
    return cleaned

for i, (text, annotations) in enumerate(training_data):
    doc = nlp.make_doc(text)
    ents = make_spans(doc, annotations, ex_id=f"ex#{i}")
    doc.ents = ents
    db.add(doc)

db.to_disk("output/train.spacy")


### Training

In [48]:
#Trainiere das Modell mit spaCy
!python3.11 -m spacy init config config.cfg --lang de --pipeline ner --optimize efficiency


✘ The provided output file already exists. To force overwriting the
config file, set the --force or -F flag.



In [49]:
!python3.11 -m spacy train config.cfg --output output --paths.train output/train.spacy --paths.dev output/train.spacy

ℹ Saving to output directory: output
ℹ Using CPU
ℹ To switch to GPU 0, use the option: --gpu-id 0

=========================== Initializing pipeline ===========================
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['tok2vec', 'ner']
ℹ Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  ------  ------
  0       0          0.00     55.11    0.00    0.00    0.00    0.00
  0     200        113.66   2279.88   86.60   90.67   82.88    0.87
  0     400        669.46    924.70   91.51   94.50   88.70    0.92
  1     600        183.16    747.65   94.48   95.45   93.53    0.94
  2     800        205.83    724.07   96.76   97.25   96.27    0.97
  2    1000        223.96    594.70   98.15   98.31   97.99    0.98
  4    1200        234.76    466.08   98.07   97.99   98.15    0.98
  5    1400        443.32    379.55   98.55   98.5

## Modelltest

In [34]:
# spaCy-Modell importieren
ner_model = spacy.load("output/model-best")

# Liste aller NER-Labels anzeigen
labels = ner_model.get_pipe("ner").labels
print(labels)

('DATE', 'LOC', 'PER')


### Test auf Textdaten

In [35]:
import spacy

In [36]:
text = """ In meinem 14ten Jahr wurde ich von einem der obgedachten Prediger confirmirt.
Im Jahr 1799 kam ich bei einem Strumpfwirker-Meister in die Lehre.
Als ich im Jahr 1802 ausgelernt hatte, beschloss ich, sogleich auf die Wanderschaft zu gehen.
In Heidelberg, wo ich nun wieder arbeitete.
In Gnadau bekam ich sogleich Arbeit auf meiner Profession.
Im März 1807 begab ich mich auf die Reise nach Herrnhut.
Am 27sten September desselben Jahres wurde ich in den Brüderbund aufgenommen.
Im Januar 1809 wurde mir angezeigt, dass ich Arbeit bekommen könnte.
Am 6 April 1815 erging der Ruf des Herrn an mich, Ihm bei der Mission in Südafrika zu dienen.
Am 25. Juli wurden wir in der Unitäts-Aeltesten-Konferenz abgefertigt.
Wir traten am folgenden Tag die Reise an.
Am 12. August langten wir in London an.
Am 30. September verließen wir London.
Am 24. December langten wir in der Capstadt an.
Am Nachmittag des 30. Decembers erreichten wir Grönekloof.
Nachdem wir mit der Hottentotten - Gemeine das Neujahrs- und Heidenfest 1816 gefeiert hatten, verließen wir Grönekloof.
Langten nach einer fünftägigen Reise in Gnadenthal an, dem nunmehrigen Ort meiner Bestimmung.
Am 3. März desselben Jahres wurde mir der Antrag gemacht, mit der ledigen Schwester Agnes Jenke in den Stand der heiligen Ehe zu treten.
Am 26. März wurden wir getraut.
Am 9. Februar 1817 wurden wir durch die Geburt eines Söhnleins erfreut.
Langten wir am 8. Mai in Enon an.
Am 20. Januar 1822 wurde mir in einer Versammlung des Hausgemeinleins eine vom Bischof Gottlob Martin Schneider ausgefertigte schriftliche Ordination zu einem Diakonus überreicht.
Anfangs Februar 1825 verließen wir Kapstadt.
Langten wir am 17. April in London an.
Am 20. Mai in Kleinwelke eintrafen.
Am 13. Juli traten wir die Rückreise nach Südafrika wieder an.
Nach einem 14-tägigen Aufenthalt da selbst begaben wir uns über Neuwied und Zeist nach London und von da nach Bedford.
Am 25. Februar 1826 langten wir nach 15 Wochen auf der stürmischen See in Kapstadt an.
Wir reisten nun über Grönekloof nach Gnadenthal.
Im Februar 1828 reisten wir zuvorderst nach der Kapstadt.
Wir verließen im März 1829 Gnadenthal.
Unser Weg führte uns zuerst nach Enon.
Am 17. August 1830 verließen wir Silo.
Nach einer zwölftägigen Reise in Enon an.
Als wir im November 1839 in Enon anlangten, sah es da selbst gar traurig aus.
Nachdem derselbe in Ebersdorf mit der ledigen Schwester Rosalie Bauer verbunden worden und diese unsere geliebten Kinder sich noch einige Zeit bei uns aufhielten, sah ich dieselben mit dankbar gebeugtem Herzen ihrem hohen Berufe entgegen gehen.
Johannes Lemmerz, heimgegangen in Kleinwelke den 6. Mai 1855."""

In [50]:
# spaCy-Modell importieren
ner_model = spacy.load("output/model-best")

doc = ner_model(text)

In [38]:
# Erkenne und gebe die Entitäten aus
for ent in doc.ents:
    print(ent.text, ent.label_)

1799 DATE
1802 DATE
Heidelberg LOC
Gnadau LOC
März 1807 DATE
Herrnhut LOC
September DATE
Januar 1809 DATE
6 April 1815 DATE
London LOC
24. December DATE
1816 DATE
Gnadenthal LOC
Agnes Jenke PER
26. März DATE
9. Februar 1817 DATE
8. Mai DATE
Enon LOC
20. Januar 1822 DATE
Gottlob Martin Schneider PER
Februar 1825 DATE
17. April DATE
London LOC
20. Mai DATE
Kleinwelke LOC
London LOC
Bedford LOC
25. Februar 1826 DATE
Kapstadt LOC
Gnadenthal LOC
Februar 1828 DATE
März 1829 DATE
Gnadenthal LOC
Enon LOC
17. August 1830 DATE
Enon LOC
November 1839 DATE
Enon LOC
Ebersdorf LOC
Rosalie Bauer PER
Johannes Lemmerz PER
Kleinwelke LOC
6. Mai DATE


### Testdatenkonvertierung

In [51]:
nlp = spacy.blank("de")

In [52]:
doc_bin = DocBin()

In [53]:
with open("test_data.jsonl", "r", encoding="utf-8") as f:
    for line in f:
        example = json.loads(line)
        doc = nlp.make_doc(example["text"])

        valid_ents = []
        occupied = set()

        for start, end, label in example["entities"]:
            if any(i in occupied for i in range(start, end)):
                continue  # überspringe überlappende Entitäten
            span = doc.char_span(start, end, label=label)
            if span:
                valid_ents.append(span)
                occupied.update(range(start, end))

        doc.ents = valid_ents
        doc_bin.add(doc)

doc_bin.to_disk("output/test.spacy")

### Testausführung

#### Ner-Modell

In [54]:
!python3.11 -m spacy evaluate output/model-best output/test.spacy --output output/metrics.json


ℹ Using CPU
ℹ To switch to GPU 0, use the option: --gpu-id 0

================================== Results ==================================

TOK     100.00
NER P   69.93 
NER R   59.30 
NER F   64.18 
SPEED   26701 


=============================== NER (per type) ===============================

           P       R       F
DATE   72.97   68.69   70.77
LOC    78.00   75.92   76.95
PER    47.99   34.79   40.34
ORG    53.33   11.68   19.16

✔ Saved results to output/metrics.json


#### spaCy de_core_news_lg Modell

In [8]:
!python3.11 -m spacy evaluate de_core_news_lg ./output/test.spacy --output output/news_metrics.json


ℹ Using CPU
ℹ To switch to GPU 0, use the option: --gpu-id 0

================================== Results ==================================

TOK      100.00
TAG      -     
POS      -     
MORPH    -     
LEMMA    -     
UAS      -     
LAS      -     
NER P    37.31 
NER R    41.85 
NER F    39.45 
SENT P   -     
SENT R   -     
SENT F   -     
SPEED    9871  


=============================== NER (per type) ===============================

           P       R       F
LOC    61.33   79.03   69.06
PER    21.87   35.79   27.15
DATE    0.00    0.00    0.00
MISC    0.00    0.00    0.00
ORG    14.05   10.76   12.19

✔ Saved results to output/news_metrics.json


In [13]:
nlp = spacy.blank("de")

In [14]:
doc_bin = DocBin()

In [4]:
with open("test_data.cleaned.jsonl", "r", encoding="utf-8") as f:
    for line in f:
        example = json.loads(line)
        doc = nlp.make_doc(example["text"])

        valid_ents = []
        occupied = set()

        for start, end, label in example["entities"]:
            if any(i in occupied for i in range(start, end)):
                continue  # überspringe überlappende Entitäten
            span = doc.char_span(start, end, label=label)
            if span:
                valid_ents.append(span)
                occupied.update(range(start, end))

        doc.ents = valid_ents
        doc_bin.add(doc)

doc_bin.to_disk("output/1_test.spacy")

In [5]:
!python3.11 -m spacy evaluate output/model-best output/1_test.spacy --output output/metrics.json

ℹ Using CPU
ℹ To switch to GPU 0, use the option: --gpu-id 0

================================== Results ==================================

TOK     100.00
NER P   49.94 
NER R   37.79 
NER F   43.02 
SPEED   27930 


=============================== NER (per type) ===============================

           P       R       F
LOC    59.50   56.14   57.77
DATE   66.67   43.14   52.39
PER    13.75   10.93   12.18
ORG     0.00    0.00    0.00

✔ Saved results to output/metrics.json
